<a href="https://colab.research.google.com/github/generalpacific/zerotohero/blob/main/zerotoherobengio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compile dataset for the neural net

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt 
# The %matplotlib inline command tells the IPython environment to draw the plots immediately after the current cell
%matplotlib inline

Read dataset in words

In [7]:
words = open('sample_data/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

Build mapping of to/from integers

In [18]:
chars = sorted(list(set(''.join(words))))

# character to integer
stoi = {s:i + 1 for i,s in enumerate(chars)}
stoi['.'] = 0

# integer to character
itos = {i:s for s,i in stoi.items()}
print(itos)



{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


Make the intial data set with the block of input and the next prediction

In [23]:
BLOCK_SIZE = 3

X = []
Y = []

for w in words[:5]:
  print(w)

  # Read the block.
  current_block = [0] * BLOCK_SIZE

  for ch in w + '.':
    i = stoi[ch]
    X.append(current_block)
    Y.append(i)
    print(''.join(itos[ib] for ib in current_block), " -> " , ch)
    current_block = current_block[1:] + [i]

  print ("\n")

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
...  ->  e
..e  ->  m
.em  ->  m
emm  ->  a
mma  ->  .


olivia
...  ->  o
..o  ->  l
.ol  ->  i
oli  ->  v
liv  ->  i
ivi  ->  a
via  ->  .


ava
...  ->  a
..a  ->  v
.av  ->  a
ava  ->  .


isabella
...  ->  i
..i  ->  s
.is  ->  a
isa  ->  b
sab  ->  e
abe  ->  l
bel  ->  l
ell  ->  a
lla  ->  .


sophia
...  ->  s
..s  ->  o
.so  ->  p
sop  ->  h
oph  ->  i
phi  ->  a
hia  ->  .




Embedding Lookup table 
* decide the embedding size


In [62]:
# Lookup table with embedding size 2.
C = torch.randn(27,2)

# embedding is simply the lookup for each input.
embedding = C[X]


MLP: First Layer
* Decide weights and number of neurons for the first layer
* challenge will be how do you multiple embedding which is `32 * 3 * 2` matrix by weights which will be `6 * number of neurons` size.

In [63]:
NUM_OF_NEURONS = 100

W1 = torch.randn(6, NUM_OF_NEURONS)
b1 = torch.randn(NUM_OF_NEURONS)

h = torch.tanh(embedding.view(embedding.shape[0], 6) @ W1 + b1)

In [46]:
h

tensor([[-1.0000,  1.0000,  0.3543,  ..., -1.0000, -0.9524, -0.9405],
        [-0.9997, -0.5203,  0.9990,  ..., -1.0000,  1.0000, -0.9230],
        [ 0.9993,  0.9993, -1.0000,  ..., -0.9926, -0.8662, -0.1419],
        ...,
        [-0.9713,  0.6152, -0.7253,  ..., -0.9419,  0.6753,  0.3512],
        [-0.9487,  0.9871, -0.8801,  ...,  0.9602, -0.8803, -0.0933],
        [-0.9993, -0.1741,  0.9915,  ..., -0.9999,  0.9375, -0.9140]])

MLP: Final layer
* Outputs logits.
* Normalize the logits so that they sum to one.

In [64]:
W2 = torch.randn(NUM_OF_NEURONS, 27)
b2 = torch.randn(27)

logits = h @ W2 + b2

# Normalize logits
counts = logits.exp()
prob = counts / counts.sum(1, keepdims = True)
prob[0].sum() 

tensor(1.0000)

Implement negative log likelihood loss
* Find the prob assigned to Y by the above NN.

In [65]:
prob_for_Y = prob[torch.arange(32), Y]

log_likelyhood = prob_for_Y.log()

mean_log_likelyhood = log_likelyhood.mean()

# negative log likelyhood
loss = -mean_log_likelyhood
loss

tensor(16.1061)

In [59]:
F.cross_entropy(logits, Y)

tensor(15.8593)

Train the model: Lets redefine parameters with same seed.


In [ ]:
g = torch.Generator().manual_seed(1232132)

# Lookup table with embedding size 2.
C = torch.randn((27, 2), generator=g)

NUM_OF_NEURONS = 100

W1 = torch.randn((6, NUM_OF_NEURONS), generator=g)
b1 = torch.randn((NUM_OF_NEURONS), generator=g)

W2 = torch.randn((NUM_OF_NEURONS, 27), generator=g)
b2 = torch.randn((27), generator=g)



In [75]:
parameters = [C, W1, b1, W2, b2]

for p in parameters:
  p.requires_grad = True

Train the model: Forward pass and backward pass

In [83]:

LEARNING_RATE = 0.1
TRAINING_PASSES = 1000

for _ in range(TRAINING_PASSES):
  # forward pass
  embedding = C[X]
  h = torch.tanh(embedding.view(-1, 6) @ W1 + b1)
  logits = h @ W2 + b2
  loss = F.cross_entropy(logits, Y)

  for p in parameters:
    p.grad = None

  # backward pass
  loss.backward()
  for p in parameters:
    p.data += (-LEARNING_RATE) * p.grad

print(loss.item())


0.2532922625541687
